## 1. Összefűzzük a dialógusokat. Létrehozunk egy file-t, amely az összes dialógust tartalmazza és egy meta file-t, hogy vissza tudjuk keresni a dialógusokat.

In [3]:
import huspacy
import os
import pickle
directory = "../data/combined_transcripts_freeConv"

docs = []
meta = {}
i = 0
for filename in os.listdir(directory):
    with open(os.path.join(directory, filename), 'r', encoding="UTF-8") as f:
        txt = f.read()

    mordor = [e.strip() for e in txt.split("\n") if e.startswith("Mordor: ")]
    gondor = [e.strip() for e in txt.split("\n") if e.startswith("Gondor: ")]

    j = len(mordor)
    k = len(gondor)
    for h in range(j):
        meta[i+h] = filename + "_mordor_" + str(h)
    for g in range(k):
        meta[i+g+j] = filename + "_gondor_" + str(g)
    mordor = [e.replace("Mordor: ", "") for e in mordor]
    gondor = [e.replace("Gondor: ", "") for e in gondor]

    docs += mordor + gondor
    a = (j+k)
    i += a
with open("../data/docs.pkl", "wb") as outfile:
    pickle.dump(docs, outfile)

with open("../data/meta.pkl", "wb") as outfile:
    pickle.dump(meta, outfile)

print("Az adatbázis",len(docs), "beszédfordulót tartalmaz.")
print("Az adatbázis első beszédfordulójának metaadata:", meta[0])

Az adatbázis 23604 beszédfordulót tartalmaz.
Az adatbázis első beszédfordulójának metaadata: pair81_freeConv_combined_srt_files.txt_mordor_0


## 2. Előfeldolgozzuk a zsöveget: lemmatizálunk, tokenizálunk, POS-tagelünk. Lementjük az adatokat. Lehet, hogy a huspacy nem fog betölteni, akkor használjuk az előre megadott lemmatizált szöveget és menjünk tovább az ebben a notebookban lévő 3. lépésre.

In [1]:
import spacy
import huspacy
nlp = huspacy.load()

In [ ]:
import pickle

docs_lemmatized = []
docs_pos = []
docs_tokenized = []

for line in docs:
    lemmatized = []
    postagged = []
    tokenized = []
    processed_line = nlp(line)
    for token in processed_line:
        if token.pos_ not in ["PUNCT"]:
            lemma = token.lemma_.replace(">","").replace(".","").lower()
            pos = f"{lemma}_{token.pos_}"
            str_token = token.text
            lemmatized.append(lemma)
            tokenized.append(str_token)
            postagged.append(pos)
    if len(lemmatized) == 0:
        lemmatized = ["U"]
    if len(postagged) == 0:
        postagged = ["U"]
    if len(tokenized) == 0:
        tokenized = ["U"]
    docs_lemmatized.append(" ".join(lemmatized))
    docs_pos.append(" ".join(postagged))
    docs_tokenized.append(" ".join(tokenized))

with open("../data/lemmatized.pkl", "wb") as outfile:
    pickle.dump(docs_lemmatized, outfile)

with open("../data/pos.pkl", "wb") as outfile:
    pickle.dump(docs_pos, outfile)

with open("../data/tokenized.pkl", "wb") as outfile:
    pickle.dump(docs_tokenized, outfile)

## 3. Folytatjuk az előfeldolgozást: kiszűrjük a stop szavakat, lementjük az adatot.

In [4]:
lemmatized = pickle.load(open("../data/lemmatized.pkl", "rb"))
with open('../data/stopwordlist.txt', 'r') as infile:
    stoplist = infile.read().split('\n')
lemmatized_split = [e.split()for e in lemmatized]
filtered = []
for e in lemmatized_split:
    temp = []
    for word in e:
        if word not in stoplist:
            temp.append(word)
    if len(temp) > 0:
        filtered.append(" ".join(temp))
    else:
        filtered.append("PlaceHolder")
with open("../data/stopword_filtered.pkl", "wb") as outfile:
    pickle.dump(filtered, outfile)